In [1]:
import numpy as np
from scipy import spatial 
import faiss
from time import time
import matplotlib.pyplot as plt
from collections import defaultdict
from sklearn.cluster import KMeans

## Helper Function

In [2]:
def semi_optimized_exhaustive_search(
        index_vectors: np.ndarray,
        query_vectors: np.ndarray,
        k: int,
):
    """
    This function performs an optimized exhaustive search.
    Args:
        index_vectors: An array of shape (n_index, dim) containing the index vectors.
        query_vectors: An array of shape (n_queries, dim) containing the query vectors. 
        dim: The dimensionality of the vectors.
    Returns:
        An array of shape (n_queries, k) containing the indices of the k nearest neighbors for each query vector.
    """
    ann_lists = []
    for query_vec in query_vectors:
        distances = np.linalg.norm(index_vectors - query_vec, axis=1)
        ann_lists.append(list(np.argsort(distances)[:k]))
    return np.array(ann_lists)

In [3]:
def build_faiss_flatl2_index(
        index_vectors: np.ndarray,
        dim: int,
):
    """
    This function builds a Faiss flat L2 index.
    Args:
        index_vectors: An array of shape (n_index, dim) containing the index vectors.
        dim: The dimensionality of the vectors. 
    Returns:
        A Faiss flat L2 index.
    """
    index = faiss.IndexFlatL2(dim)
    index.add(index_vectors)
    return index

In [4]:
def faiss_search(
        query_vectors: np.ndarray,
        index: faiss.Index,
        k: int,
):
    """
    This function uses a Faiss index to search for the k-nearest neighbors of query_vectors.
    Args:
        query_vectors: An array of shape (n_queries, dim) containing the query vectors. 
        index: A Faiss index.
        k: The number of nearest neighbors to retrieve.
    Returns:
        An array of shape (, ) containing the indices of the k-nearest neighbors for each query vector.
    """
    distances, indices = index.search(query_vectors, k)
    return indices

In [5]:
def build_faiss_lsh_index(
        index_vectors: np.ndarray,
        dim: int,
        nbits: int,
):
    """
    This function builds a Faiss LSH index.
    Args:
        index_vectors: An array of shape (n_index, dim) containing the index vectors.
        dim: The dimensionality of the vectors. 
        nbits: The number of bits to use in the hash.
    Returns:
        A Faiss LSH index.
    """
    index = faiss.IndexLSH(dim, nbits)
    index.add(index_vectors)
    return index

In [6]:
def compute_recall_at_k(
        nn_gt: np.ndarray,
        ann: np.ndarray,
        k: int,
):
    """
    This function computes the recall@k.
    Args:
        nn_gt: The ground truth nearest neighbors.
        ann: The approximate nearest neighbors.
        k: The number of nearest neighbors to consider.
    Returns:
        The recall@k.
    """
    return round(sum([len(set(ann[i]) & set(nn_gt[i])) / k for i in range(len(ann))])/len(ann), 3)

# 2.1 -- LSH vs Naive Exhaustive Search (Regular Index Vectors)
### You just have to run the following cells and add the following results to the report:
* running time of the ground truth computation with semi_optimized_exhaustive_search (wall time)
* running time of creating faiss_lsh_index (wall time)
* running time of faiss_search over query_vectors with faiss_lsh_index (wall time)
* recall@10 for faiss_lsh_ann

In [7]:
query_vectors = np.load('data/query_vectors.npy')
index_vectors = np.load('data/index_vectors.npy')
k=10
dim = index_vectors.shape[1]

In [8]:
%%time
gt_nn = semi_optimized_exhaustive_search(index_vectors, query_vectors, k)

CPU times: total: 24.5 s
Wall time: 25.9 s


In [9]:
%%time
faiss_lsh_index = build_faiss_lsh_index(index_vectors, dim, nbits=2000)

CPU times: total: 2.81 s
Wall time: 1.56 s


In [10]:
%%time
faiss_lsh_ann = faiss_search(query_vectors, faiss_lsh_index, k)

CPU times: total: 1.98 s
Wall time: 697 ms


In [11]:
faiss_lsh_ann.shape

(1000, 10)

In [12]:
print(f"recall@10 for faiss_lsh_index: {compute_recall_at_k(gt_nn, faiss_lsh_ann, k)}")

recall@10 for faiss_lsh_index: 0.138


# 2.2 -- Custom Indexing Algorithm
Build an indexing algorithm that satisfies the following requirements:
* The indexing algorithm should be able to handle vectors of different dimensions
* The running time of the indexing should be less than half of the running time of semi_optimized_exhaustive_search), reported in Section 2.1.
* The running time of searching over the index should be less than a third (1/3) of the time of the semi_optimized_exhaustive_search function, reported in Section 2.1.
* The performance (in terms of recall@10) of the indexing algorithm should be at least 0.8.

The last three bullets should also appear in the report.
You are allowed to add as many helper functions as you need. You cannot use faiss of scipy libraries for this task. Numpy is allowed. 

You can also test your algorithm with the additional two query-index sets by replacing the calls made few cells ago to:

    query_vectors = np.load('data/query_vectors2.npy')
    index_vectors = np.load('data/index_vectors2.npy')
or:

    query_vectors = np.load('data/query_vectors3.npy')
    index_vectors = np.load('data/index_vectors3.npy')
    
the aforementioned requirements should also be satisfied over these two query-index sets. No need to insert the results over these two to the report.

In [13]:
#TODO: Write your code for 2.2.2 here
# You are allowed to add more arguments to the functions and create more functions if needed.

def update_k_means_centroids(vectors, centroids):
    """
    Update the k-means centroids
    """
    n_clusters = len(centroids)
    distances = np.linalg.norm(vectors[:, np.newaxis] - centroids, axis=2)
    labels = np.argmin(distances, axis=1)
    new_centroids = []
    for cluster_id in range(n_clusters):
        cluster_vectors = vectors[labels == cluster_id]
        # remove empty clusters
        if len(cluster_vectors) > 0:
            new_centroid = cluster_vectors.mean(axis=0) 
            new_centroids.append(new_centroid)
        
    return np.array(new_centroids)

def separate_to_clusters(vectors, centroids):
    """
    Given vectors and centroids, separate the vectors to an index based on the centroids
    :return: The vectors divided into clusters, the vector ids divided into clusters
    """
    distances = np.linalg.norm(vectors[:, np.newaxis] - centroids, axis=2)
    labels = np.argmin(distances, axis=1)
    clusters_vectors = {}
    clusters_vector_ids = {}
    for vector_id, (vector, label) in enumerate(zip(vectors, labels)):
        if label in clusters_vectors:
            clusters_vectors[label].append(vector)
            clusters_vector_ids[label].append(vector_id)
        else:
            clusters_vectors[label] = [vector,]
            clusters_vector_ids[label] = [vector_id,]
            
    for label in clusters_vectors:
        clusters_vectors[label] = np.array(clusters_vectors[label])
        clusters_vector_ids[label] = np.array(clusters_vector_ids[label])
            
    return clusters_vectors, clusters_vector_ids

def initialize_k_means_pp(vectors, n_clusters):
    """
    Initialize k-means-plus-plus clusters
    """
    # Choose the first centroid uniformly
    # Choose vectors to be centroids,
    # with bigger probability the farther they are from previously chosen centroids 
    centroids = np.zeros((n_clusters, vectors.shape[1]))
    centroids[0] = vectors[np.random.choice(vectors.shape[0])]  # Choose the first centroid randomly
    for i in range(1, n_clusters):
        distances = np.zeros(vectors.shape[0])
        for j, vector in enumerate(vectors):
            distances[j] = np.min(np.linalg.norm(vector - centroids[:i], axis=1))
        probabilities = distances**2 / np.sum(distances**2)
        next_centroid_idx = np.random.choice(np.arange(len(vectors)), p=probabilities)
        centroids[i] = vectors[next_centroid_idx]
    return centroids
    
def calc_k_means_clusters(vectors, n_clusters, max_iter=100):
    """
    Calculate k-means-plus-plus clusters
    :param n_clusters: The number of clusters
    :param max_iter: Max iterations number
    :return: The centroids, the vectors divided into clusters, the vector ids divided into clusters
    """
    iter = 0
    centroids = initialize_k_means_pp(vectors, n_clusters)
    prev_centroids = centroids
    centroids = update_k_means_centroids(vectors, centroids)
    while iter < max_iter and not np.array_equal(centroids, prev_centroids):
        prev_centroids = centroids
        centroids = update_k_means_centroids(vectors, centroids)
        iter += 1

    clusters_vectors, clusters_vector_ids = separate_to_clusters(vectors, centroids)
    
    return centroids, clusters_vectors, clusters_vector_ids


def custom_indexing_algorithm(index_vectors, dim, n_clusters = 8):
    """
    This function builds an index from scratch.
    Args:
        index_vectors: An array of shape (n_index, dim) containing the index vectors.
        dim: The dimensionality of the vectors.
    Returns:
        An index.
    """
    centroids, clusters_vectors, clusters_vector_ids = calc_k_means_clusters(index_vectors, n_clusters)
    return centroids, clusters_vectors, clusters_vector_ids


def custom_index_search(query_vectors, index, k):
    """
    This function searches over the custom index.
    Args:
        query_vectors: An array of shape (n_queries, dim) containing the query vectors.
        index: The custom index.
        k: The number of nearest neighbors to retrieve.
    """
    centroids, clusters_vectors, clusters_vector_ids = index
    query_centroid_distances = np.linalg.norm(query_vectors[:, np.newaxis] - centroids, axis=2)
    labels = np.argmin(query_centroid_distances, axis=1)
    
    ann_lists = []
    
    for query_vector, label in zip(query_vectors, labels):
        cluster_index_vectors = clusters_vectors[label]
        cluster_index_vectors_ids = clusters_vector_ids[label]
        
        distances = np.linalg.norm(cluster_index_vectors - query_vector, axis=1)
        ann_lists.append(cluster_index_vectors_ids[np.argsort(distances)[:k]])
        
    return np.array(ann_lists)
    

In [14]:
# Add hyperparameters here (if needed)

In [15]:
%%time
custom_index = custom_indexing_algorithm(index_vectors, dim)

CPU times: total: 5.61 s
Wall time: 6.61 s


In [16]:
%%time
custom_index_ann = custom_index_search(query_vectors, custom_index, k)

CPU times: total: 5.78 s
Wall time: 7.36 s


In [17]:
print(f"recall@10 for custom_index_search: {compute_recall_at_k(gt_nn, custom_index_ann, k)}")

recall@10 for custom_index_search: 1.0
